In [13]:
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_models import ChatOllama
from langgraph.graph import StateGraph, START, END
from langchain_community.chat_models import ChatOllama


In [14]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]


In [15]:
llm = ChatOllama(
    model="qwen3:1.7b",      # change to your model name
    temperature=0,       # deterministic output
    base_url="http://localhost:11434"  # Ollama default server URL
)

In [16]:
def process(state: AgentState) -> AgentState:
    response = llm.invoke(state['messages'])
    state['messages'].append(AIMessage(content=response.content))
    print(f"Response: {response.content}")
    return state

In [17]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)
agent = graph.compile()


In [18]:
conversation_history = []

In [19]:
user_input = input("Enter : ")
while user_input.lower() != "exit":
    conversation_history.append(HumanMessage(content=user_input))
    result = agent.invoke({
        "messages": conversation_history
    })
    print(result["messages"])
    conversation_history = result["messages"]
    user_input = input("Enter : ")

Response: <think>
Okay, the user said "hi". I need to respond in a friendly and helpful way. Let me start by acknowledging their greeting. Maybe mention that I'm here to help. Keep it simple and open-ended so they feel comfortable asking questions. Add an emoji to keep it friendly. Check if there's anything specific they need, but don't push too hard. Just a welcoming message.
</think>

Hello! I'm here to help. How can I assist you today? 😊
[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='<think>\nOkay, the user said "hi". I need to respond in a friendly and helpful way. Let me start by acknowledging their greeting. Maybe mention that I\'m here to help. Keep it simple and open-ended so they feel comfortable asking questions. Add an emoji to keep it friendly. Check if there\'s anything specific they need, but don\'t push too hard. Just a welcoming message.\n</think>\n\nHello! I\'m here to help. How can I assist you today? 😊', additional_kwargs=